In [1]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List
from datasets import load_dataset
import tensorflow as tf
from gensim.models import fasttext
from textsim_class import TextSimilarity
import spacy
from gensim.models import KeyedVectors
from gensim.models import word2vec


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("projecte-aina/sts-ca", trust_remote_code=True)

In [3]:
"""
from gensim.models import FastText
import gensim
# No funciona de moment
model = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')
"""

"\nfrom gensim.models import FastText\nimport gensim\n# No funciona de moment\nmodel = gensim.models.fasttext.FastTextKeyedVectors.load('./models/fasttext/cc_ca_300.bin')\n"

In [3]:
wv_model = word2vec.Word2Vec.load(f"./models/word2vec_complete.model")
wv_model = wv_model.wv

In [21]:
model = spacy.load("ca_core_news_trf")

In [14]:
model("hola que tal")

hola que tal

In [20]:
a = TextSimilarity(model=model, dataset=dataset, mode="roberta", pretrained=True, rob=False, remap=True, trainable=True, dict_size=1000000)

TypeError: 'KeyedVectors' object is not callable

In [18]:
a.define_model(id=0)

c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_12      │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 96, 100)   │  1,312,600 │ input_layer_11[0… │
│ (Embedding)         │                   │            │ input_layer_12[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_6          │ (None, 96)        │          0 │ input_layer_11[0… │
│ (MyLayer)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_7          │ (None, 96)        │          0 │ input_layer_12[0… │
│ (MyLayer)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 100)       │          0 │ embedding_4[0][0… │
│ (GlobalAveragePool… │                   │            │ my_layer_6[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 100)       │          0 │ embedding_4[1][0… │
│ (GlobalAveragePool… │                   │            │ my_layer_7[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 100)       │          0 │ global_average_p… │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 100)       │          0 │ lambda_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │      1,616 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 16)        │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         17 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 1)         │          0 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,314,233 (5.01 MB)

 Trainable params: 1,314,233 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
pearson_train, pearson_val, pearson_test = a.train(num_epochs=3)

Epoch 1/3
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.7348 - val_loss: 0.7240
Epoch 2/3
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.7241 - val_loss: 0.7205
Epoch 3/3
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.7131 - val_loss: 0.7157
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Correlación de Pearson (train): 0.4376983227319968
Correlación de Pearson (validation): 0.1951321159408268
Correlación de Pearson (test): 0.2277289207470134


In [11]:
a.baseline_model()

Correlación de Pearson (baseline-train): 0.17008905266923566
Correlación de Pearson (baseline-validation): 0.20186650376345033
Correlación de Pearson (baseline-test): 0.2054049066530338


(0.17008905266923566, 0.20186650376345033, 0.2054049066530338)

In [ ]:
print(pearson_train, pearson_val, pearson_test )

In [27]:
from scipy.stats import pearsonr
def compute_pearson(x_, y_):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    y_pred = a.exec_model.predict(x_)
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation
print(f"Correlación de Pearson (baseline-train): {compute_pearson(a.x_train, a.y_train)}")
print(f"Correlación de Pearson (baseline-validation): {compute_pearson(a.x_val, a.y_val)}")
print(f"Correlación de Pearson (baseline-test): {compute_pearson(a.x_test, a.y_test)}")



65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Correlación de Pearson (baseline-train): 0.6944484626760427
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-validation): 0.2903849157142713
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (baseline-test): 0.3850071528624018


In [ ]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)


c:\Users\Usuario\Documents\Universitat\4rt Quatri\PLH\Practica4\word-embeddings\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--projecte-aina--roberta-base-ca-v2-cased-sts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[{'label': 'SIMILARITY', 'score': 2.1183004307689126}, {'label': 'SIMILARITY', 'score': 2.179974932297432}, {'label': 'SIMILARITY', 'score': 0.9511617858568939}]


In [ ]:
input_pairs = [(e["sentence1"], e["sentence2"]) for e in dataset["train"]]
y_train = [e["label"] for e in dataset["train"]]
input_pairs_val = [(e["sentence1"], e["sentence2"]) for e in dataset["validation"]]
y_val = [e["label"] for e in dataset["validation"]]
input_pairs_test = [(e["sentence1"], e["sentence2"]) for e in dataset["test"]]
y_test = [e["label"] for e in dataset["test"]]

In [ ]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep



def compute_pearson(input, y):
    # Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
    predictions = pipe(prepare(input), add_special_tokens=False)

    for prediction in predictions:
        prediction['score'] = logit(prediction['score'])

    y_pred = np.array([pred['score']for pred in predictions])
    # Calcular la correlación de Pearson entre las predicciones y los datos de prueba
    correlation, _ = pearsonr(y_pred.flatten(), np.array(y).flatten())
    return correlation

print(f"Correlación de Pearson (baseline-test): {compute_pearson(input_pairs_test, y_test)}")

Correlación de Pearson (baseline-test): 0.781988582023819
